In [ ]:
import numpy as np
import scipy.ndimage as ndimage
import scipy.integrate

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.colors

import time
import math

%matplotlib inline

In [ ]:
fe_diffusion_kernel = np.array(
         [[[0, 0, 0],
           [0, 1, 0],
           [0, 0, 0]],
            
          [[0,  1, 0],
           [1, -6, 1],
           [0,  1, 0]],
            
          [[0, 0, 0],
           [0, 1, 0],
           [0, 0, 0]]]);

def convolve(domain, kernel, dr):
    return ndimage.convolve(domain, kernel / (dr ** 2), mode='constant', cval=1)
def convolve_1d(domain, kernel, shape, dr):
    return convolve(domain.reshape(shape),kernel,dr).reshape((-1,))

def diffusion_two_phase(domain, phase_mask, partition_coeff, d_interior, d_exterior, dr):
    # To simulate a partition coefficent, divide inner phase by partition_coeff
    # before doing convolution
    modified_domain = domain.copy();
    modified_domain[phase_mask] /= partition_coeff;
    
    result = convolve(modified_domain, fe_diffusion_kernel, dr)
    # Scale back changes in interior by partition_coeff
    result[phase_mask] *= partition_coeff * d_interior;
    result[~phase_mask] *= d_exterior;
    return result;

def diffusion_1d(domain, phase_mask, partition_coeff, d_interior, d_exterior, dr, shape):
    return diffusion_two_phase(domain.reshape(shape), phase_mask, partition_coeff, d_interior, d_exterior, dr).reshape((-1,))

In [ ]:
def generate_packed_sphere_mask(domain_shape, num_spheres, r_sphere, r_centers):
    # Generates a hexagonal close packing of spheres, separated by r_centers
    num_rows = math.ceil(math.sqrt(num_spheres))
    entries_per_row = math.ceil(num_spheres / num_rows)
    horz_sep = r_centers;
    horz_offset = int(horz_sep / 2);
    vert_offset = int(math.sqrt(3) / 2 * r_centers);
    
    x_offsets = [horz_sep * i for i in range(entries_per_row)];
    avg_offset = sum(x_offsets) / entries_per_row;
    x_offsets.sort(key=lambda x: abs(x - avg_offset))
    
    x_start = domain_shape[0] / 2 - (horz_sep * (-1/2 + entries_per_row / 2));
    y_start = domain_shape[1] / 2 - (vert_offset * (-1/2 + num_rows / 2));
    
    total_mask = np.zeros(domain_shape, dtype=bool)
    n_placed = 0;
    for i in range(num_rows):
        for x_offset in x_offsets:
            if n_placed < num_spheres:
                total_mask |= generate_sphere_mask(domain_shape,
                           int(x_offset + x_start + (horz_offset * (i % 2))),
                           int(y_start + vert_offset * i),
                           int(domain_shape[2] / 2),
                           r_sphere);
            n_placed += 1
    return total_mask

In [ ]:
def generate_sphere_mask(domain_shape,x,y,z,r):
    """
    Given an (x,y,z) sphere point (in index units) and r (in index units),
    returns a boolean mask over the domain where true values lie within the sphere.
    """
    x_vals = np.arange(domain_shape[0]).reshape(-1,1,1);
    y_vals = np.arange(domain_shape[1]).reshape(1,-1,1);
    z_vals = np.arange(domain_shape[2]).reshape(1,1,-1);
    return ((x_vals - x)**2 + (y_vals - y)**2 + (z_vals - z)**2) < r**2

def run_simulation(diffusion_interior, diffusion_exterior, partition_coeff,
                 num_spheres, t_max, domain_resolution):
    """
    Uses a 
    Inputs:
        
    """
    domain_shape = (domain_resolution * 5, domain_resolution * 5, domain_resolution)
    width_spheres = int(domain_resolution * .7)
    ics = np.ones(domain_shape);
    sphere_mask = generate_packed_sphere_mask(ics.shape, num_spheres, width_spheres / 2, int(1.3 *width_spheres))
    ics[sphere_mask] = 0;
    
    def ivp_func(t,x,mask=sphere_mask,shape=domain_shape,
                 di=diffusion_interior, de=diffusion_exterior, K=partition_coeff):
        return diffusion_1d(x, mask, K, di, de, 1, shape)
    
    results = scipy.integrate.solve_ivp(ivp_func, [0, t_max], ics.reshape((-1,)), method='RK23');
    assert(results.success);
    return (results.y.reshape(domain_shape + (results.y.shape[1],)),sphere_mask)

In [ ]:
pre_wall, pre_cpu = (time.perf_counter(), time.process_time())
domain_solution, phase_mask = run_simulation(.3,1,2,10,30,45)
post_wall, post_cpu = (time.perf_counter(), time.process_time())
print('Wall time: {:.2f} sec, CPU time: {:.2f}'.format(post_wall - pre_wall, post_cpu - pre_cpu))

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(10,5))
im = axes[0].imshow(domain_solution[:,:,22,0], 'gray',matplotlib.colors.Normalize(0,2), aspect='equal')
mean_recovery = [np.mean((domain_solution[:,:,:,i])[phase_mask]) for i in range(domain_solution.shape[3])]
recovery_curve, = axes[1].plot(mean_recovery)
axes[1].set_title('Mean recovery inside spheres')
point, = axes[1].plot(0,mean_recovery[0], 'o')
axes[1].plot(mean_recovery, 'k', linewidth=.5, alpha=.4)
def animate_func(i):
    im.set_array(domain_solution[:,:,22,i])
    recovery_curve.set_xdata(range(i))
    recovery_curve.set_ydata(mean_recovery[:i])
    point.set_data(i, mean_recovery[i])
anim = FuncAnimation(fig,animate_func,frames=domain_solution.shape[3])
anim.save('test.mp4')